In [1]:
import pandas as pd
import xarray as xr
import geopandas as gpd
import numpy as np
import os
import re
import datetime
import matplotlib.pyplot as plt
from rasterio.plot import show
from skimage.filters.rank import majority
from tqdm.notebook import tqdm
from sklearn.neighbors import BallTree
from matplotlib.colors import ListedColormap, BoundaryNorm

%config Completer.use_jedi = False
%matplotlib widget

from distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=5, threads_per_worker=2)
client = Client(cluster)

In [2]:
"""
Define input data and covariates for extraction
"""
# set the year of analysis
year = 2018

# path to input directory
inDIR = 'C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/data/'
# file name of gridded path intensity data (output from gps_to_gridded_path_intensity.ipynb)
griddata_f = str(year) + '_grazing_time_gridded_all.csv'

# create dictionary of paths to dynamic time-series co-variates
ts_covariates = {
    'Biomass': {
        'path': 'C:/SPK_local/data/rasters/HLS/HLS_CARM_biomass/CPER_biomass_pred_simp_' + str(year) + '.tif'},
    'CP': {
        'path': 'C:/SPK_local/data/rasters/HLS/HLS_quality/CPER_CP_' + str(year) + '.tif'},
    'DOM': {
        'path': 'C:/SPK_local/data/rasters/HLS/HLS_quality/CPER_DOM_' + str(year) + '.tif'}
}

# create dictionary of paths to static co-variates
static_covariates = {
    'TPC': {
        'path': 'C:/SPK_local/data/rasters/DEM/TopoClass25m.tif'},
    'dFence': {
        'path': 'C:/SPK_local/data/rasters/Masks/CPER_dist_to_fence_2017.tif'},
    'dTank': {
        'path': 'C:/SPK_local/data/rasters/Masks/CPER_dist_to_tank_2017.tif'}
}

# path to 1 m vegetation raster
veg_f = 'G:/neon_v18/neon_class_2017_v18.tif'
#veg_f = 'T:/3-GIS/CPER/Layers/NEON/veg_maps_v18/neon_class_2017_v18.tif'

# path to shapefile containing pasture corners
corners_f = 'C:/SPK_local/data/vectors/CPER_features/CPER_pasture_corners.shp'

In [3]:
"""
Create dictionaries for remapping raster integer values to character strings
"""
# create dictionary to remap plant community values to strings
veg_dict = {'C4': [0],
            'Annual': [1],
            'Bare_veg': [2, 3], # combines Bare_veg with FORB
            'C3': [4, 5], # combines HECO and PASM
            'C3_C4_mix': [6, 7],
            'Saltgrass': [8],
            'Shrub': [9],
            'Bare': [10],
            'UNK': [11, 255]
            }

# create dictionary to remap topgoraphic position class (TPC) values to strings
TPC_dict = {
    1.0: 'Lowlands', 2.0: 'Lowlands', 3.0: 'Lowlands',
    4.0: 'Other',
    5.0: 'Flat Plains',
    6.0: 'Open Slopes', 
    7.0: 'Other', 
    8.0: 'Highlands', 9.0: 'Highlands', 10.0: 'Highlands'
}

In [4]:
"""
Convert gridded covariates to xarray datasets
"""
for i in ts_covariates:
    print(i)
    xr_tmp = xr.open_rasterio(ts_covariates[i]['path'])
    xr_tmp = xr_tmp.rename({'band': 'date'})
    xr_tmp['date'] = [datetime.datetime(2017, 1, 1) + 
                   datetime.timedelta(d-1) for d in xr_tmp['date'].astype('float').values]
    ts_covariates[i]['data'] = xr_tmp

for i in static_covariates:
    print(i)
    xr_tmp = xr.open_rasterio(static_covariates[i]['path']).squeeze('band')
    if i == 'TPC':
        xr_tmp = xr_tmp.astype('int')
    static_covariates[i]['data'] = xr_tmp

Biomass
CP
DOM
TPC
dFence
dTank


In [5]:
"""
Prepare plant community (veg) covariates to be added to static covariates
"""
# load 1 m vegetation raster to lazy xarray DataAarray
xr_veg = xr.open_rasterio(veg_f).squeeze('band')#.chunk({'y': 300, 'x': 300})
# rename the DataArray
xr_veg.name = 'VEG_FG'

# run majority filter multiple times to change NAN values to the majority nearest-neighbour value
print('Running majority filters')
print('1 of 3')
img_maj1 = majority(xr_veg.values, np.ones((3, 3)))
xr_veg1 = xr_veg.where(xr_veg != 255, other=img_maj1)
print('2 of 3')
img_maj2 = majority(xr_veg1.values, np.ones((3, 3)))
xr_veg2 = xr_veg1.where(xr_veg1 != 255, other=img_maj2)
print('3 of 3')
img_maj3 = majority(xr_veg2.values, np.ones((3, 3)))
xr_veg3 = xr_veg2.where(xr_veg2 != 255, other=img_maj3)
print('Majority filters finished!')

Running majority filters
1 of 3
2 of 3
3 of 3
Majority filters finished!


In [6]:
"""
Plot the vegetation data after each filtering iteration to determine final filtering level
"""
fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(6, 6), sharex=True, sharey=True)
cmap_veg = ListedColormap(['blue', 'orange', 'red', 'green', 'lightseagreen', 'pink', 'yellow', 'brown', 'black'])
norm = BoundaryNorm([0, 1, 2, 4, 6, 8, 9, 10, 11], cmap_veg.N, clip=True)
show(xr_veg, cmap=cmap_veg, norm=norm, ax=axs.flatten()[0])
show(xr_veg1, cmap=cmap_veg, norm=norm, ax=axs.flatten()[1])
show(xr_veg2, cmap=cmap_veg, norm=norm, ax=axs.flatten()[2])
show(xr_veg3, cmap=cmap_veg, norm=norm, ax=axs.flatten()[3])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [7]:
"""
Save filtered vegetation maps as GeoTIFFs for preview in GIS if desired
"""
# from rasterio.crs import CRS
# import rasterio as rio
# out_profile = {'driver': 'GTiff',
#                        'dtype': 'int16',
#                        'nodata': -999,
#                        'width': len(xr_veg.x),
#                        'height': len(xr_veg.y),
#                        'count': 1,
#                        'crs': CRS.from_dict(init='epsg:32613'),
#                        'transform': rio.Affine(1.0, 0.0, np.min(xr_veg.x - 0.5).data,
#                                                0.0, -1.0, np.max(xr_veg.y + 0.5).data),
#                        'tiled': False}
# with rio.open('G:/neon_v18/neon_class_2017_v18_maj1.tif',
#               'w', **out_profile) as dst:
#     dst.write(xr_veg1.data[np.newaxis, :, :])
# with rio.open('G:/neon_v18/neon_class_2017_v18_maj2.tif',
#           'w', **out_profile) as dst:
#     dst.write(xr_veg2.data[np.newaxis, :, :])
# with rio.open('G:/neon_v18/neon_class_2017_v18_maj3.tif',
#           'w', **out_profile) as dst:
#     dst.write(xr_veg3.data[np.newaxis, :, :])

'\nSave filtered vegetation maps as GeoTIFFs for preview in GIS if desired\n'

In [8]:
"""
Calculate the plant community metrics to be extracted
"""

# specify the final filtered vegetation map to use as a covariate
xr_veg = xr_veg3

# create a 30 m coarsened DataArray of counts 1 m cells in each 30 m cell
xr_veg_30m = xr_veg.coarsen(dim=dict(x=30, y=30), boundary='trim').count()
# convert DataArray to Dataset
xr_veg_30m = xr_veg_30m.to_dataset()

# loop through each plant community class, convert counts to a proportion, rename the class and add it as a variable
for k in veg_dict:
    print(k)
    # get count of 1 m cells for the individual class in the 30 m cell
    xr_veg_30m = xr_veg_30m.assign(TMP=(['y', 'x'],
                                  xr_veg.where(xr_veg.isin(veg_dict[k])).coarsen(dim=dict(x=30, y=30),
                                                                           boundary='trim').count()))
    # convert the count to a proportion based on the total number of 1 m cells
    xr_veg_30m['TMP'] = xr_veg_30m['TMP'] / xr_veg_30m['VEG_FG']
    # rename the class based on the dictionary
    xr_veg_30m = xr_veg_30m.rename({'TMP': k})

# 'snap' the 30 m plant community data to the coordinates of the other 30 m gridded datasets
xr_veg_30m = xr_veg_30m.sel(y=static_covariates['dFence']['data']['y'],
                        x=static_covariates['dFence']['data']['x'],
                        method='nearest').reindex_like(static_covariates['dFence']['data'], 
                                                       method='nearest')
# replace any NAN values with zeros
xr_veg_30m = xr_veg_30m.fillna(0)

# identify the dominant plant community class in each 30 m cell as the class with the greatest proportion
static_covariates['PC_dmt'] = {'data':
                               xr_veg_30m[[i for i in veg_dict]].to_array().idxmax('variable')}
# calculate the proprotional coverage of the dominant class
static_covariates['PC_pct'] = {'data':
                               xr_veg_30m[[i for i in veg_dict]].to_array().max('variable')}
# calculate the Shannon diversity index of all classes in the 30 m grid cell
static_covariates['PC_div'] = {'data':
                              (xr_veg_30m[[i for i in veg_dict]].to_array() * 
                               xr.ufuncs.log(xr_veg_30m[[i for i in veg_dict]].to_array().where(
                                   xr_veg_30m[[i for i in veg_dict]].to_array() != 0))).sum('variable') * -1.0}

C4
Annual
Bare_veg
C3
C3_C4_mix
Saltgrass
Shrub
Bare
UNK


In [9]:
"""
Sanity check of vegetation metrics by plotting
"""
# create an integer version of the dominant vegetation classes for plotting
xr_PC_dmt_int = static_covariates['PC_dmt']['data'].copy()
xr_PC_dmt_int.values = np.array([veg_dict[i][0] for i in static_covariates['PC_dmt']['data'].values.flatten()]).reshape(xr_PC_dmt_int.shape)

# plot the three vegetation metrics together for comparison
fig, axs = plt.subplots(ncols=3, figsize=(9, 4), sharex=True, sharey=True)
show(xr_PC_dmt_int, cmap='tab10', ax=axs[0])
show(static_covariates['PC_pct']['data'], cmap='viridis', ax=axs[1])
show(static_covariates['PC_div']['data'], cmap='magma', ax=axs[2])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [10]:
"""
Loop through static and time series co-variates and extract for each grid cell
"""
# read in the gridded path intensity dataset
df_wkly_grid = pd.read_csv(os.path.join(inDIR, griddata_f), engine='python')

for g in tqdm(df_wkly_grid.groupby('week')):
    target_lon = xr.DataArray(g[1]['UTM_X'], dims="points")
    target_lat = xr.DataArray(g[1]['UTM_Y'], dims="points")
    # loop through static covariates and get values
    for var in static_covariates:
        xr_tmp = static_covariates[var]['data']
        df_wkly_grid.loc[df_wkly_grid['week'] == g[0], var] = xr_tmp.sel(
            x=target_lon, y=target_lat, method='nearest').values
    # loop through time series covariates and get weekly average
    for var in ts_covariates:
        xr_tmp = ts_covariates[var]['data']
        df_wkly_grid.loc[df_wkly_grid['week'] == g[0], var] = xr_tmp[xr_tmp.date.dt.isocalendar().week == g[0]].mean('date').sel(
            x=target_lon, y=target_lat, method='nearest').values
        

  0%|          | 0/21 [00:00<?, ?it/s]

In [11]:
"""
Get distance to nearest pasture corner for each pasture
"""
# read in shapefile of corners
gdf_corners = gpd.read_file(corners_f)

# loop through pastures and get the distance from each fix to the pasture corner
for past_i in tqdm(df_wkly_grid['Pasture'].unique()):
    #print(past_i)
    gdf_corners_p = gdf_corners[gdf_corners['Pasture'] == past_i]
    x_coords = [i[0][0] for i in gdf_corners_p.apply(lambda x: x.geometry.coords.xy, axis=1)]
    y_coords = [i[1][0] for i in gdf_corners_p.apply(lambda x: x.geometry.coords.xy, axis=1)]

    # Create a BallTree object
    # borrowed from: https://stackoverflow.com/questions/58893719/find-nearest-point-in-other-dataframe-with-a-lot-of-data
    tree = BallTree(np.array(list(zip(x_coords, y_coords))), leaf_size=2)

    #Query the BallTree on all GPS coordinates from collars deployed in the pasture to find the distance
    # to the nearest pixel within the pasture and its id
    dist_tmp, id_tmp= tree.query(
        df_wkly_grid.loc[df_wkly_grid['Pasture'] == past_i, ['UTM_X', 'UTM_Y']].values, # The input array for the query
        k=1, # The number of nearest neighbors
    )
    df_wkly_grid.loc[df_wkly_grid['Pasture'] == past_i, 'dCorner'] = dist_tmp

  0%|          | 0/10 [00:00<?, ?it/s]

In [12]:
"""
Make sure all variables are in the correct datatype format
"""
# convert topographic position class (TPC) values to strings
df_wkly_grid['TPC_c'] = [TPC_dict[c] for c in df_wkly_grid['TPC']]

# print the datatypes for each field
display(df_wkly_grid.dtypes)

index_id          int64
mod_data         object
week              int64
Pasture          object
Steer_ID         object
UTM_X           float64
UTM_Y           float64
grazing_secs    float64
TPC             float64
dFence          float64
dTank           float64
PC_dmt           object
PC_pct          float64
PC_div          float64
Biomass         float32
CP              float32
DOM             float32
dCorner         float64
TPC_c            object
dtype: object

In [13]:
"""
Write gridded data with extrations to disk
"""
df_wkly_grid.to_csv(os.path.join(inDIR, re.sub('.csv', '_extracted.csv', griddata_f)), index=False)